In [15]:
# PACKAGES FOR TRANSLATION
!pip install transformers -q
!pip install sentencepiece -q
!pip install sacremoses -q

# # PACKAGES FOR SENTENCE VECTORIZERS
!pip install -U sentence-transformers -q
!pip install faiss-cpu -q
!pip install langchain -q

# PACKAGES FOR LLM
!pip install -q datasets loralib sentencepiece -q
!pip uninstall transformers -q
!pip install -q git+https://github.com/zphang/transformers@c3dc391 -q
!pip -q install git+https://github.com/huggingface/peft.git -q
!pip -q install bitsandbytes -q

Proceed (Y/n)? Y
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [16]:
from trans import tr2eng ### tr2eng translation task
# from trans import tr2eng
from sent_vectorizer import vectorizer, cleaner ### sentence vectorizer
import faiss
import numpy as np 
import pandas as pd
from langchain.prompts import PromptTemplate

In [17]:
# eng_trans = tr2eng('benim adim mehmet')
# print(tr2eng('Turkiye\'nin baskenti Ankara\'dir.'))

In [61]:
# Who is Beyoncé Giselle Knowles-Carter? What is her profession?
query = 'Beyoncé Giselle Knowles-Carter kimdir ve meslegi nedir?' 

In [62]:
query_eng = tr2eng(query)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [63]:
print(query_eng)

Who is Beyoncé Giselle Knowles-Carter and what is her profession?


In [64]:
eng_query_vector = vectorizer(query_eng)
eng_query_vector = np.array(eng_query_vector).reshape(1,384)

In [65]:
vector_store = faiss.read_index("drive/MyDrive/QQ_PROJECTS/turkish_project/squad_turkish/squad_turkish/squad_index.index")

In [66]:
K = 3
top_index = vector_store.search(np.array(eng_query_vector), k =3)[1][0][0]

In [67]:
df =  pd.read_csv('drive/MyDrive/QQ_PROJECTS/turkish_project/squad_turkish/squad_turkish/squad_unique_tr_eng.csv')

In [68]:
top_index

0

In [69]:
query_context =  df['english'][top_index]

In [70]:
from langchain.prompts import PromptTemplate

In [71]:
input_prompt = PromptTemplate(
    input_variables = ['given_context','given_question'],
    template = """try to answer the given question according to the given text.
        Given text : {given_context}
        Question : {given_question}
        """
)

In [72]:
query_context

" Beyoncé Giselle Knowles-Carter (born September 4, 1981) is an American singer, songwriter, producer and actress. She was born and raised in Houston, Texas, performed in various singing and dancing contests as a child, and was awarded the R&B girls' R&B band Destiny's Child in the late 1990s, and was named as the lead singer of her father, Mathew Knowles, one of the best-selling girls of all time.</s>"

In [73]:
context = """Beyoncé Giselle Knowles-Carter (born September 4, 1981) is an American singer, songwriter, producer and actress. 
            She was born and raised in Houston, Texas, performed in various singing and dancing contests as a child, 
            and was awarded the R&B girls\' R&B band Destiny\'s Child in the late 1990s, and was named as the lead singer of her father, 
            Mathew Knowles, one of the best-selling girls of all time. 
            """
question ="""when is the birthday of Beyoncé Giselle Knowles-Carter?"""             
LLM_input = input_prompt.format(given_context = query_context, given_question = query_eng)

In [74]:
print(LLM_input)

try to answer the given question according to the given text.
        Given text :  Beyoncé Giselle Knowles-Carter (born September 4, 1981) is an American singer, songwriter, producer and actress. She was born and raised in Houston, Texas, performed in various singing and dancing contests as a child, and was awarded the R&B girls' R&B band Destiny's Child in the late 1990s, and was named as the lead singer of her father, Mathew Knowles, one of the best-selling girls of all time.</s>
        Question : Who is Beyoncé Giselle Knowles-Carter and what is her profession?
        


In [59]:
from llm_starter import alpaca_talk

In [33]:
cont = """  
            Given text :
            Beyoncé Giselle Knowles-Carter (born September 4, 1981) is an American singer, songwriter, producer and actress. 
            She was born and raised in Houston, Texas, performed in various singing and dancing contests as a child, 
            and was awarded the R&B girls\' R&B band Destiny\'s Child in the late 1990s, and was named as the lead singer of her father, 
            Mathew Knowles, one of the best-selling girls of all time. 

            try to answer the given question according to the given text.
            Question :
            when is the birthday of Beyoncé Giselle Knowles-Carter?
            """
pref = """
        try to answer the given question according to the given text.
        """

q = """
    Question :
    when is the birthday of Beyoncé Giselle Knowles-Carter?
    """        

In [34]:
import re
re.sub('    ',' ',cont)

"  \n   Given text :\n   Beyoncé Giselle Knowles-Carter (born September 4, 1981) is an American singer, songwriter, producer and actress. \n   She was born and raised in Houston, Texas, performed in various singing and dancing contests as a child, \n   and was awarded the R&B girls' R&B band Destiny's Child in the late 1990s, and was named as the lead singer of her father, \n   Mathew Knowles, one of the best-selling girls of all time. \n\n   try to answer the given question according to the given text.\n   Question :\n   when is the birthday of Beyoncé Giselle Knowles-Carter?\n   "

In [77]:
alpaca_res = alpaca_talk("""  
            Given text :
            Beyoncé Giselle Knowles-Carter (born September 4, 1981) is an American singer, songwriter, producer and actress. 
            She was born and raised in Houston, Texas, performed in various singing and dancing contests as a child, 
            and was awarded the R&B girls\' R&B band Destiny\'s Child in the late 1990s, and was named as the lead singer of her father, 
            Mathew Knowles, one of the best-selling girls of all time. 

            try to answer the given question according to the given text.
            given question :
            Who is Beyoncé Giselle Knowles-Carter? What is her profession?
            And translate the answer into turkish language.
            """)

Generating...
   
            Given text :
            Beyoncé Giselle Knowles-Carter (born September 4, 1981) is an American singer, songwriter, producer and actress. 
            She was born and raised in Houston, Texas, performed in various singing and dancing contests as a child, 
            and was awarded the R&B girls' R&B band Destiny's Child in the late 1990s, and was named as the lead singer of her father, 
            Mathew Knowles, one of the best-selling girls of all time. 

            try to answer the given question according to the given text.
            given question :
            Who is Beyoncé Giselle Knowles-Carter? What is her profession?
            And translate the answer into turkish language.
            
        """
        
        # get input from user
        question = self._get_question()
        answer = self._get_answer(question)
    
        if not answer:
            return None
                
        # convert english sentence to turkish sen

In [75]:
alpaca_res = alpaca_talk(LLM_input)

Generating...
 try to answer the given question according to the given text.
        Given text :  Beyoncé Giselle Knowles-Carter (born September 4, 1981) is an American singer, songwriter, producer and actress. She was born and raised in Houston, Texas, performed in various singing and dancing contests as a child, and was awarded the R&B girls' R&B band Destiny's Child in the late 1990s, and was named as the lead singer of her father, Mathew Knowles, one of the best-selling girls of all time.</s>
        Question : Who is Beyoncé Giselle Knowles-Carter and what is her profession?
         Answer: Beyonce Gisele Knowles Carter is an American Singer, Song Writer, Producer & Actress
